In [8]:

# play sentences in Anki
# initial date: 2015-11-20

# to-do: add image field? "crock" in "chili served in a crock"
# to-do: use utf-8 instead of ascii for puntuations like dash, quotations, and foreign words


import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import re

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest


#fdict = open('anki\\sPlayAnki.txt', 'w')
fdict = codecs.open('anki\\sentPlayAnki.txt', 'w', 'utf-8')

# read list of words with images [captions] inot imglist
with codecs.open('anki\\imglist.txt', 'r', 'utf-8') as f:
    imglist = f.read().splitlines() # strip '\n'
    f.close()
  
    
#results = geweiDict.find({"fr":"y"}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, 
#                              "idpvs":1, "sublist":1, "audiofile":1, "alles":1})
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, 
                              "idpvs":1, "sublist":1, "audiofile":1, "alles":1, "fr":1,
                              "tag":1, "core3":1, "label":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0

# threshold of the length of definition
defthres = 12


# Produce Anki note fields and tags:
for result in results:
    
    word = result.get("hw")    
    if word:
        #word = hw
        #word = hw.encode('ascii', 'ignore')
        #word = hw.encode('utf-8')
        print word
        totalhw += 1
    else:
        print "no head word"
        continue

    
    # get items of result
    core3 = result.get('core3')
    tagindict = result.get('tag')
    label = result.get('label')
    
    infls = result.get("infl")
    subhws = result.get("subhw")   
    
    homograph = 0
    audiofile = result.get("audiofile")
    
    idpvs = result.get("idpvs")
    sublists = result.get("sublist") #sublists is a list whose items are dictionary
    
    fr = result.get("fr")

    alles = result.get("alles")        
    if alles: totales = len(alles)
    else: totales=0
        
    if totales >= 100: nes = "100+"
    elif totales >= 50: nes = "50-99"
    elif totales >= 10: nes = "10-49"
    elif totales >=  3: nes = "3-9"
    elif totales >=  2: nes = "2"
    elif totales >=  1: nes = "1"
    else: nes = "0"
        
    
    if sublists:
        cnt = 1
        idiom = 0
        
        # each "es" or "idpv" of sublist in sublists will be an Anki note
        for sublist in sublists: #sublist is dictionary

            # Anki note field: PlaySent
            if sublist.has_key("es"):                
                # no example sentence but have definition, write head word to PlaySent field
                if len(sublist["es"]) == 0 and len(sublist["defi"]) >= defthres:
                    #print defthres, " --- ", word                    
                    fdict.write(word)
                # this es is empty and "def" is not a definition, discard this fake definition
                elif len(sublist["es"]) == 0 and len(sublist["defi"]) < defthres:
                    print word, ": empyt example and not a defintion"
                    continue                    
                else:
                    for sPlay in sublist["es"]:                    
                        #sPlay = es.encode('ascii', 'ignore').replace(word, "___")                        
                        #sPlay = es
                        insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                        sPlay = insensitive_replace.sub('___', sPlay)
                        
                        if infls:
                            for infl in infls:
                                sPlay = sPlay.replace(infl, "...")
                        if subhws:
                            for subhw in subhws:
                                sPlay = sPlay.replace(subhw, "???")
                        fdict.write(str(cnt) + '<font color="blue">-es: </font>' + sPlay + "<br>")
                    
            if sublist.has_key("idpv"):
                if len(sublist["idpv"]) == 0:
                    print word, ": empyt idpv"
                    continue
                #sPlay = sublist["idpv"].encode('ascii', 'ignore').replace(word, "___")
                #sPlay = sublist["idpv"].encode('ascii', 'ignore')
                sPlay = sublist["idpv"]
                sPlay = re.sub(r'\s+(?=\d)', '<br><br>', sPlay) # replace the space before a number with '<br><br>'
                
                insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                sPlay = insensitive_replace.sub('___', sPlay)
                if sPlay.find(":") >= 0: idiom = 1
                
                if infls:
                    for infl in infls:
                        sPlay = sPlay.replace(infl, "...")
                if subhws:
                    for subhw in subhws:
                        sPlay = sPlay.replace(subhw, "???")
                fdict.write('<font color="blue">*idpv*</font> ' + sPlay + "<br>")
            #if cnt%1 == 0: fdict.write('<br>')
            fdict.write(" \t")            
            
            
            # Anki note field: head word
            fdict.write(word + "\t")           
            
            # Anki note field: Pronunciation in ASCII IPA symbols
            prns = result.get("prn")
            if prns:
                for prn in prns:
                    fdict.write("/" + prn + "/<br>")
            fdict.write(" \t")  # add one space before tab to separate tabs in case a field is empty. 
            

            # Anki note field: Audio    
            if word: 
                if audiofile: 
                    fdict.write("[sound:" + audiofile + ".wav]")
                    homograph = 1
                else: fdict.write("[sound:" + word + ".wav]")
            if subhws:
                for subhw in subhws:
                    #fdict.write("<br>[sound:" + subhw.encode('ascii', 'ignore') + ".wav]")
                    fdict.write("<br>[sound:" + subhw + ".wav]")
            fdict.write(" \t")    


            # Anki note field: definition
            if sublist.has_key("defi"):                
                #fdict.write(str(cnt) + '-def: ' + sublist["def"].encode('ascii', 'ignore') + "<br><br>")
                fdict.write(str(cnt) + '-def: ' + sublist["defi"] + "<br><br>")            
            fdict.write(" \t")
                
            # Anki note field: Wow
            if sublist.has_key("es"):  
                for es in sublist["es"]:
                    fdict.write(str(cnt) + '<font color="red">-es: </font>' + 
                                es + "<br>")                
                                #es.encode('ascii', 'ignore') + "<br>")                
            if sublist.has_key("idpv"):                
                fdict.write('<font color="red">*idpv*</font> ' + 
                            re.sub(r'\s+(?=\d)', '<br><br>', sublist["idpv"]) + "<br>")
                            #sublist["idpv"].encode('ascii', 'ignore') + "<br>")
            #if cnt%1 == 0: fdict.write('<br>') 
            fdict.write(" \t")
            
               
            tags = "splay"
            if core3: tags += ' core3'
            if tagindict:
                for tag in tagindict:
                    tags += ' ' + tag
            if label:
                if 'slang' in label: tags += ' slang'
                if 'baseball' in label: tags += ' baseball'
                
            if idiom == 1: tags += " idiom"
            if fr: tags += " fr" # Foreign word (French, Mexican Spanish)       
            if homograph == 1 and word != 'con' : tags += " homog" # homog -- homograph
            tags += " es" + nes # level of a word by the number of examples
            
            # Anki note field: Footnote            
            fdict.write("<br><br>{" + tags + "}<br><br>\t")
            
            # Anki note fields: Image and Caption
            img = ' '
            caption = ' '
            imgflag = 0
            for imgname in imglist:
                if word == imgname.split('.')[0]: 
                    imgl = imgname.split(';')
                    print imgl
                    img += '<img src="' + imgl[0] + '" />'
                    imgflag = 1
                    if len(imgl) == 2: caption += imgl[1]
                    elif len(imgl) == 3: caption += imgl[1] + ';' + imgl[2]
                    
            fdict.write(img + "\t")
            fdict.write(caption + "\t")

            # Anki tags
            if imgflag == 1: tags += " img"
            fdict.write(tags + "\n")
            
            cnt += 1
    
fdict.close()
print "total head words: ", totalhw





yard
yeah
yeah : empyt example and not a defintion
X-ray
y
Yankee
Yankee : empyt example and not a defintion
yard
yoo-hoo
yoof
yore
Yorkshire terrier
you'd
you'll
you're
you-all
you-know-who
youngster
young gun
yucca
yuck
Yule
Yuletide
yummy
yup
yuppie
yuppify
Y chromosome
zaftig
zany
zapper
zapper : empyt example and not a defintion
zealous
zebra
[u'zebra.gif']
zebra crossing
zeitgeist
zenith
zephyr
zeppelin
zero-sum game
zero
zero gravity
zero hour
zero tolerance
zesty
zigzag
[u'zigzag.gif']
zigzag
[u'zigzag.gif']
zilch
zillion
Zimmer frame
zinc
zine
zing
zing
zing : empyt example and not a defintion
zinger
Zionism
zip
zip
zip
zip
zippo
zippy
zippy : empyt example and not a defintion
zip code
zit
zither
zodiac
[u'zodiac.gif']
[u'zodiac.gif']
zombie
zonal
zone defense
zookeeper
zoological park
zoology
zoom lens
zoot suit
Zoroastrianism
zucchini
Zulu
zwieback
zygote
con
without
workaday
workday
worker
workforce
workings
working class
working papers
working party
working week
workload
w

In [7]:
# this snippets of codes organize images list to a unique sorted list
# first add new items in imglistraw.txt, the snippets will create imglist.txt which will be used in codes above.

import codecs

fimg = codecs.open('anki\\imglist.txt', 'w', 'utf-8')
with codecs.open('anki\\imglistraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()

cnt = 0    
uImgs = sorted(set(lines))

for img in uImgs:    
    fimg.write(img + '\n')
    cnt += 1
    print cnt, ': ', img
fimg.close()


1 :  Alabama.png
2 :  Alaska.png
3 :  Arizona.png
4 :  Arkansas.png
5 :  California.png
6 :  Colorado.png
7 :  Connecticut.png
8 :  Delaware.png
9 :  Ferris wheel.gif
10 :  Florida.png
11 :  Georgia.png
12 :  Hawaii.png
13 :  Idaho.png
14 :  Illinois.png
15 :  Indiana.png
16 :  Iowa.png
17 :  Kansas.png
18 :  Kentucky.png
19 :  Louisiana.png
20 :  Maine.png
21 :  Maryland.png
22 :  Massachusetts.png
23 :  Michigan.png
24 :  Minnesota.png
25 :  Mississippi.png
26 :  Missouri.png
27 :  Montana.png
28 :  Nebraska.png
29 :  Nevada.png
30 :  New-Hampshire.png
31 :  New-Jersey.png
32 :  New-Mexico.png
33 :  New-York.png
34 :  North-Carolina.png
35 :  North-Dakota.png
36 :  Ohio.png
37 :  Oklahoma.png
38 :  Oregon.png
39 :  Pekingese.gif
40 :  Pennsylvania.png
41 :  Rhode-Island.png
42 :  South-Carolina.png
43 :  South-Dakota.png
44 :  Tennessee.png
45 :  Texas.png
46 :  Utah.png
47 :  Vermont.png
48 :  Virginia.png
49 :  Washington.png
50 :  West-Virginia.png
51 :  Wisconsin.png
52 :  Wordie

In [11]:
a = [1, 2, 3, 4, 5]
a[1:]

[2, 3, 4, 5]

In [6]:
word="abc\nu0x99999"
print word
print word.encode('ascii', 'ignore')

abc
u0x99999
abc
u0x99999


In [2]:
tstr="we adjust oursleves to the weather"
print tstr.replace("adjust", "___")

we ___ oursleves to the weather


In [25]:
sPlay = "You mean he lied to us? Bingo!"
repl = "bingo"
resub = re.compile(re.escape(repl), re.IGNORECASE)
print resub.sub("___", sPlay) # sPlay doesn't change
print sPlay

You mean he lied to us? ___!
You mean he lied to us? Bingo!


In [5]:
with codecs.open('anki\\imagelist.txt', 'r', 'utf-8') as f:
    imglist1 = f.read().splitlines() # strip '\n'
with codecs.open('anki\\imagelist.txt', 'r', 'utf-8') as f:    
    imglist2 = f.readlines() # include '\n'

print imglist1[0:5]
print imglist2[0:5]


[u'aardvark.gif', u'abacus.gif', u'abscissa.gif', u'accordion.gif', u'airplane.gif']
[u'aardvark.gif\n', u'abacus.gif\n', u'abscissa.gif\n', u'accordion.gif\n', u'airplane.gif\n']


In [9]:
tl = ['f1.jpg','f2.gif', 'f3.png']
word = 'f2'
for tt in tl:
    if word == tt.split('.')[0]: print 'yeah, it is ', tt
    else: print 'not in the list'

not in the list
yeah, it is  f2.gif
not in the list


In [12]:
tl = ['f1.jpg','f2.gif', 'f3.png']
for tt in tl:
    tt += ': image!' # don't change tl
    print tt
print tl

f1.jpg: image!
f2.gif: image!
f3.png: image!
['f1.jpg', 'f2.gif', 'f3.png']


In [6]:
import re

idpv1 ="set up [phrasal verb] 1 : to prepare for something by putting things where they need to be We have one hour left to set up for the party. We had better start setting up. 2 set up (something) or set (something) up 2 a : to create or establish (something) for a particular purpose A panel needs to be set up to investigate the issue. She hopes that in a few years she can set up a private practice of her own. A foundation was set up for people with the disease. The church set up a school for the city's homeless children. The hikers set up camp [=they prepared an area where they could sleep outside] just before sunset. 2 b : to arrange and plan (an event or activity) I'll ask my secretary to set up a meeting with our clients. Let's set up a time to get together. He was the one who set up the bank robbery. 2 c : to put (something) in an upright position They set up flags along the street for the parade. Help me set the tent up. 2 d : to make (a machine, system, etc.) ready to be used They set up the movie camera next to the door. This computer wasn't set up to run so many programs at one time. 3 set (someone) up or set up (someone) 3 a : to cause (someone) to be in a bad situation or to appear guilty Those aren't his drugs. Someone must have set him up! [=framed him; made it look like the drugs were his] He claimed he was set up by the police. 3 b Brit : to help (someone) feel healthy and full of energy She says that daily exercise has set her up to face the challenges of getting old. 3 c : to give (someone) a job, a place to live, etc. She set me up in a great apartment in the city. He set his son up with a job at his company. 3 d : to do something that makes it likely or possible for (someone) to do, get, or experience something The team's excellent defense set them up to score the winning touchdown. usually + for I think you're just setting yourself up for a big disappointment. [=you're expecting something that won't happen and you will be disappointed when it doesn't] 3 e : to cause (someone) to meet someone in order to start a romantic relationship How did you first meet your husband? My best friend set us up. often + with She wants to set him up with her sister. 4 set (yourself) up as (something) 4 a : to try to make people believe that you are (something) She has set herself up as the best alternative to the current administration. He sets himself up as a defender of the people, but he's really only interested in getting more power. 4 b also set up as (something) or set up in (something) : to start your own business She set (herself) up as a wedding planner. [=she began working as a wedding planner] He set up in business after finishing college. see also SETUP"
idpv2 ="*idpv* take up [phrasal verb] 1 take up or take up (something) or take (something) up : to begin (something) again : to continue (something) after you or another person stops She took up [=picked up] the story where he left off. He is hoping that he and his ex-girlfriend can take up [=that they can continue their relationship] where they left off. 2 take up (something) : to fill (an area, amount of time, etc.) completely or almost completely The new couch takes up half of the room. I don't want to take up too much of your time, but I do have a few questions. The entire day was taken up by/with meetings. We don't use this table for anything. It's just taking up space. 3 take up (something) or take (something) up 3 a : to begin studying or practicing (an activity, subject, instrument, etc.) usually as a hobby I was thinking about taking up skiing/dancing/photography. She took up the guitar at age 11. taking up art lessons 3 b : to begin to deal with (a problem, an issue, etc.) The cause of global warming has been taken up by many celebrities recently. The court took up the question of how to deal with companies that break the law. He seemed willing to take up [=take on] the challenge. 3 c : to begin to have (a new job, home, etc.) He will take up his post [=begin working at his new post/job] at the beginning of the year. She went to France and took up residence in Paris. [=became a resident of Paris; began living in Paris] She took up [=she began to lead] the life of an artist. = She took up life as an artist. Two men with guns had taken up (their) positions on the roof. 3 d somewhat old-fashioned : to begin to use (something) They took up hammers and nails and went to work building the house. Once again, they took up arms [=picked up weapons and became ready to fight] to defend their country. 3 e : to make (something, such as a piece of clothing) shorter Can you take the legs of these pants up two inches? 3 f : to lift and remove (something) We took up the carpet in the living room and replaced it with hardwood flooring. 3 g : to gather (money, clothes, etc.) from many different people or places They are taking up a collection for the homeless shelter. 4 take (someone) up on (something) : to make an agreement with (someone) to accept (an offer) Can I buy you a drink? Sure, I'll take you up on that. We took the company up on its offer to replace the computer for free. 5 take (something) up with (someone) : to talk about (something, such as a problem) with (someone) If you have a problem, please take it up with one of our managers. Have you taken this up with your mother yet? 6 take up with (someone) : to begin a friendly or romantic relationship with (someone) After her divorce, she took up with a younger man. see also TAKE-UP"

idpv1 = re.sub(r'\s+(?=\d)', '\n', idpv1)
print idpv1
print '------'
tt2 = re.sub(r'\s+(?=\d)', '\n', idpv2)
print tt2

set up [phrasal verb]
1 : to prepare for something by putting things where they need to be We have one hour left to set up for the party. We had better start setting up.
2 set up (something) or set (something) up
2 a : to create or establish (something) for a particular purpose A panel needs to be set up to investigate the issue. She hopes that in a few years she can set up a private practice of her own. A foundation was set up for people with the disease. The church set up a school for the city's homeless children. The hikers set up camp [=they prepared an area where they could sleep outside] just before sunset.
2 b : to arrange and plan (an event or activity) I'll ask my secretary to set up a meeting with our clients. Let's set up a time to get together. He was the one who set up the bank robbery.
2 c : to put (something) in an upright position They set up flags along the street for the parade. Help me set the tent up.
2 d : to make (a machine, system, etc.) ready to be used They set

In [6]:
import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import re

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511


fdict = codecs.open('anki\\fr.txt', 'w', 'utf-8')
    
results = geweiDict.find({"fr":"y"}, {"_id":0, "hw":1})

# Produce Anki note fields and tags:
for result in results:    
    hw = result.get("hw")
    print hw
    fdict.write(hw + "\n")
    
fdict.close()    

éclair
élan
El Niño
émigré
entrée
exposé
fiancé
fiancée
flambé
appliqué
fraîche
après-ski
Gruyère
habitué
attaché
attaché case
jalapeño
lamé
macramé
manqué
matériel
mélange
ménage à trois
métier
née
outré
papier-mâché
passé
pâté
pièce de résistance
piña colada
piñata
précis
protégé
protégée
raison d'être
recherché
bête noire
résumé
risqué
rosé
sauté
sauté
séance
blasé
soufflé
soupçon
tête-à-tête
bon appétit
touché
transport café
über-
vicuña
voilà
brûlée
café
canapé
cause célèbre
célèbre
chargé d'affaires
château
cliché
communiqué
à la
à la carte
à la mode
consommé
coup de grâce
coup d'état
crèche
crème anglaise
crème brûlée
crème caramel
crème de la crème
crème de menthe
crème fraîche
crudités
cybercafé
décolletage
déjà vu
détente
divorcé
divorcée
doppelgänger
